# Python Interface Quickstart



**SarcAsM** is a Python package for computational analysis of sarcomere structure and motion in cardiomyocytes. In this quickstart tutorial a general workflow including all functionalities is presented. Function parameters are explained in docstrings. Further documentation can be found in package API reference. SarcAsM processes single images as well as movies.

In [ ]:
# import package
from sarcasm.package import *

In [ ]:
# select tif file and initialize SarcAsM object
tif_file = '../test_data/Series32_20kPa.tif'
sarc_obj = SarcAsM(tif_file, restart=False)

In [ ]:
# predict sarcomere z-bands with (Siam-)U-Net CNN
network_path = '../trained_networks/siam_unet_sarcomeres.pth'
sarc_obj.predict_z_bands(siam_unet=True, network_path=network_path, size=(256, 1024))

## Structure analysis

1. Segmentation and morphometric analysis of sarcomere z-bands
2. Wavelet analysis of local sarcomere lengths and orientations
3. Myofibril line growth algorithm for myofibril lengths and curvature analysis

By default all frames are analyzed and features are computed for each frame (``timepoints='all'``). To analyze only one frame or a range of frames, choose e.g.,``timepoints=10``, ``timepoints=[2,5,87]`` or ``timepoints=np.arange(100)``. 

All results are stored in dictionary ``sarc_obj.structure``. The dictionary is saved as .json file to the project folder after each analysis step. 

#### 1. Segmentation and morphometric analysis of sarcomere z-bands

In [ ]:
sarc_obj.analyze_z_bands(timepoints=0)

#### 2. Wavelet analysis of local sarcomere lengths and orientations

In [ ]:
sarc_obj.wavelet_analysis(timepoints=0)

#### 3. Myofibril line growth algorithm for myofibril lengths and curvature analysis

``timepoints`` are inherited from step 2.

In [ ]:
sarc_obj.analyze_myofibrils(timepoints=0, n_seeds=200)

To run steps 1-3 at once with default parameters, use ``sarc_obj.fast_analysis_structure()``.

## Motion analysis

0. Automatically detect line ROIs for sarcomere motion tracking
1. Detect peaks from kymographs
2. Track z-band positions
3. Detect and analyze systoles from average sarcomere length
4. Calculate sarcomere displacements and velocities
5. Analyze trajectories

#### Detect ROI lines for motion analysis

In [ ]:
sarc_obj.detect_rois(timepoint=0, plot=True)

### Select ROI

In [ ]:
# find all ROI files of cell
roi_files = glob.glob(sarc_obj.folder + '*.json')

# select ROI from list
i = 1

# initialize object
roi_file = os.path.basename(roi_files[i])
roi = Motion(tif_file, roi_file)

### Detect peaks

In [ ]:
# find peaks of intensity profiles
roi.detekt_peaks(thres=0.05)

### Track z-band trajectories

In [ ]:
# track peaks with Crocker-Grier algorithm
roi.track_z_bands()

### Detect systoles with neural network and analyze contraction cycles

In [ ]:
# smoothing of z-band trajectories
roi.filter_z_pos()

In [ ]:
roi.detect_analyze_systoles(plot=True, contr_time_min=0.15, merge_time_max=0.025, threshold=0.4, perc=None)

### Calculate sarcomere resting lengths, displacements and velocities

In [ ]:
roi.get_sarcomere_trajectories()

### Analyze sarcomere trajectories (min and max in each contraction cycle)

In [ ]:
roi.analyze_sarcomere_trajectories()

### Analyze sarcomere popping

In [ ]:
roi.analyze_popping()

### Plot summary of ROI

In [ ]:
plot_roi_summary_motion(roi, number_contr=4, t_lim=(-0.1, 2.3))